# Decision Trees

Reference: [An Introduction to Statistical Learning](http://www-bcf.usc.edu/~gareth/ISL/) (This is one of the must-read books for data scientists)

**Decision trees** are _non-parametric_ models that can either be used for regression or classification. They are the building blocks for **random forest** and **xgboost** models, which are often used to win machine learning competitions. You are likely very familiar with decision trees already! For example, a rule-based engine in your organization.

**Random forests** are groups of decision trees created using different subsets and feature sets of the training data. Each tree "votes" on a classification or "predicts" on a regression task. We will cover it later.

### Why are we learning about decision trees?

- They can be applied to both regression and classification problems.
- They are easy to explain to others (interpretable).
- They are very popular among data scientists.
- They are the basis for more sophisticated models.
- They have a different way of "thinking" than the other models we have studied.

## Regression Trees - Numeric Output

For instance, suppose we wanted to predict Major League Baseball salaries from 1986–87 using the following data set.

- **Years** (x-axis): Number of years playing in the major leagues.
- **Hits** (y-axis): Number of hits in the previous year.
- **Salary** (color): Low salary is blue/green, high salary is red/yellow.

![Salary data](./img/salary_color.png)

If you plotted these data in three dimensions, with salary corresponding to height, *linear regression* would find the plane that goes through the heart of the data (minimizes the sum of squared distances from the data points).

A **regression tree** takes a different approach. It breaks the 2D plane shown into segments according to the following rules:

- You can only use **straight lines** that are drawn one at a time.
- Your line must either be **vertical or horizontal**.
- Your line **stops** when it hits an existing line.

It then predicts the average value within a segment for every point in that segment.

For instance, it might start with a vertical line that separates most of the purple points from the rest:

![](./img/salary_color_split1.png)

Next it might split between veterans with high and low numbers of hits:

![](./img/salary_color_split2.png)

In general, instead of a plane, a decision tree regression model gives you "stair steps."

![](./img/tree_3d_plot.png)

Here is another way to visualize the model:

![Salary tree annotated](./img/salary_tree_annotated.png)

The first split is **years < 4.5**, thus that split goes at the top of the tree. When a splitting rule is **true**, you follow the left branch. When a splitting rule is **false**, you follow the right branch.

For players in the **left branch**, the mean salary is \$166,000, thus you label it with that value. (Salary has been divided by 1,000 and log-transformed to 5.11.)

For players in the **right branch**, there is a further split on **hits < 117.5**, dividing players into two more salary regions: \$403,000 (transformed to 6.00), and \$846,000 (transformed to 6.74).

**Note:** Years and hits are both integers, but the convention is to use the **midpoint** between adjacent values to label a split.

**What does this tree tell you about your data?**



## Building Decision Trees for Classification Tasks

In [ ]:
# let's get a new dataset again
!mkdir ./data/breast_cancer_wisconsin
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.data > ./data/breast_cancer_wisconsin/breast-cancer-wisconsin.data
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/breast-cancer-wisconsin/breast-cancer-wisconsin.names > ./data/breast_cancer_wisconsin/breast-cancer-wisconsin.names

In [ ]:
!cat ./data/breast_cancer_wisconsin/breast-cancer-wisconsin.names

Requirements

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.tree import DecisionTreeClassifier
%matplotlib inline

In [ ]:
fpath = './data/breast_cancer_wisconsin/breast-cancer-wisconsin.data'
colnames = ['id', 'clump_thickness', 'uniformity_of_cell_size', 'uniformity_of_cell_shape',
            'marginal_adhesion', 'single_epithelial_cell_size', 'bare_nuclei',
            'bland_chromatin', 'normal_nucleoli', 'mitoses', 'target']

In [ ]:
df = pd.read_csv(fpath, names=colnames)

# transform a string value in `bare_nuclei` into unknown
df['bare_nuclei'] = df.bare_nuclei.apply(lambda x: 0 if not x.isnumeric() else int(x))
df.dropna(inplace=True)
df.reset_index(drop=True, inplace=True)

In [ ]:
df.head()

- Explore the data by sorting, plotting, or performing split-apply-combine (a.k.a. `group_by`). Decide which feature you think is the most important predictor, and use it to create a splitting rule. (Only binary splits are allowed.)

In [ ]:
y = df.loc[:, 'target']

In [ ]:
for var in colnames[1:-1]:
    print(var)
    x = df.loc[:, var]
    plt.scatter(x, y)
    plt.show()

What do you think?

Surprisingly, this type of non-linear, meaning non-linearly-separable, problems are perfect for tree based methods, for which we will begin learning from decision tree algorithms.

## How Does a Computer Build a Regression Tree?

**Ideal approach:** Consider every possible partition of the feature space.

**Problem:** Too many possibilities to consider.

**"Good enough" approach:** Recursive binary splitting.

1. Begin at the top of the tree.
2. For **every feature**, examine **every possible cutpoint**, and choose the feature and cutpoint so that the resulting tree has the lowest possible error. Make that split.
3. Examine the two resulting regions. Once again, make a **single split** (in one of the regions) to minimize the MSE.
4. Keep repeating Step 3 until a **stopping criterion** is met:
    - Maximum tree depth (maximum number of splits required to arrive at a leaf).
    - Minimum number of observations in a leaf.

---

This approach is a **greedy algorithm** because it makes *locally optimal* decisions -- it takes the best split at each step. Greedy algorithms typically are not optimal, but they are often good enough and relatively easy to compute.

**Analogy:**
- Always eating cookies to maximize your immediate happiness (greedy) might not lead to optimal overall happiness.
- In our case, reorganizing parts of the tree already constructed based on future splits might result in a better model overall. 

## Building a Classification Tree in `scikit-learn`

In [ ]:
treeclf = DecisionTreeClassifier(random_state=123)
treeclf

**what does our target variable look like?**

In [ ]:
df.target.value_counts() # 2 for benign, 4 for malignant

Let's first transform the label into 0 and 1, representing benign and malignant.

In [ ]:
df['target'] = df.target.map({2:0, 4:1})

In [ ]:
df.target.value_counts()

In [ ]:
df.head()

In [ ]:
# Define X and y.
feature_cols = colnames[1:-1]
X = df.loc[:, feature_cols]
y = df.loc[:, 'target']

In [ ]:
# Use leave-one-out cross-validation (LOOCV) to estimate the precision for this model.
# we will discuss how to train models properly later, where we will touch on LOOCV
from sklearn.model_selection import cross_val_score
scores = cross_val_score(treeclf, X, y, scoring='precision')
np.mean(scores)

`Precision` evaluates how accurate the predicted positive cases are. For example, if a model predicts 10 instances to be positive, and 9 of them are actually positive, the precision is 0.9.

In sum, the process of builiding a decision tree for classification task in Scikit-Learn is very similar to that to build linear regression and logistic regression. We go through three steps:

    1. Define predictors (X) and predicted variable or target(Y)
    2. Instantiate the model
    3. "Fit" the model

## Building a Regression Tree

In [ ]:
# let's get a new dataset again
!curl https://archive.ics.uci.edu/ml/machine-learning-databases/00332/OnlineNewsPopularity.zip > ./data/OnlineNewsPopularity.zip
!unzip ./data/OnlineNewsPopularity.zip -d ./data/
!rm ./data/OnlineNewsPopularity.zip

Here are the path to the data:

    Data path: ./data/OnlineNewsPopularity/OnlineNewsPopularity.csv
    Meta Data Path: ./data/OnlineNewsPopularity/OnlineNewsPopularity.names  

In [ ]:
!cat ./data/OnlineNewsPopularity/OnlineNewsPopularity.names

**Exercise**

Try use the above example to build a regression tree for this dataset.

More about the dataset: [documentation](https://archive.ics.uci.edu/ml/datasets/Online+News+Popularity)

In [ ]:
fpath = './data/OnlineNewsPopularity/OnlineNewsPopularity.csv'

In [ ]:
df = pd.read_csv(fpath)

In [ ]:
df.head()

## Comparing Regression Trees and Classification Trees

|Regression Trees|Classification Trees|
|---|---|
|Predict a continuous response.|Predict a categorical response.|
|Predict using mean response of each leaf.|Predict using most commonly occurring class of each leaf.|
|Splits are chosen to minimize MSE.|Splits are chosen to minimize Gini index (discussed below).|

## Splitting Criteria for Classification Trees

Common options for the splitting criteria:

- **Classification error rate:** The fraction of training observations in a region that don't belong to the most common class.
- **Gini index:** The measure of total variance across classes in a region.

### Example: Classification Error Rate

Pretend we are predicting whether or not someone will buy an iPhone or an Android:

- At a particular node, there are **25 observations** (phone buyers) of whom **10 bought iPhones and 15 bought Androids**.
- As the majority class is **Android**, that's our prediction for all 25 observations, and thus the classification error rate is **10/25 = 40%**.

Our goal in making splits is to **reduce the classification error rate**. Let's try splitting on gender:

- **Males:** 2 iPhones and 12 Androids, thus the predicted class is Android.
- **Females:** 8 iPhones and 3 Androids, thus the predicted class is iPhone.
- Classification error rate after this split would be **5/25 = 20%**.

Compare that with a split on age:

- **30 or younger:** 4 iPhones and 8 Androids, thus the predicted class is Android.
- **31 or older:** 6 iPhones and 7 Androids, thus the predicted class is Android.
- Classification error rate after this split would be **10/25 = 40%**.

The decision tree algorithm will try **every possible split across all features** and choose the one that **reduces the error rate the most.**

### Example: Gini Index

Calculate the Gini index before making a split:

$$1 - \left(\frac {iPhone} {Total}\right)^2 - \left(\frac {Android} {Total}\right)^2 = 1 - \left(\frac {10} {25}\right)^2 - \left(\frac {15} {25}\right)^2 = 0.48$$

- The **maximum value** of the Gini index is 0.5 and occurs when the classes are perfectly balanced in a node.
- The **minimum value** of the Gini index is 0 and occurs when there is only one class represented in a node.
- A node with a lower Gini index is said to be more "pure."

Evaluating the split on **gender** using the Gini index:

$$\text{Males: } 1 - \left(\frac {2} {14}\right)^2 - \left(\frac {12} {14}\right)^2 = 0.24$$
$$\text{Females: } 1 - \left(\frac {8} {11}\right)^2 - \left(\frac {3} {11}\right)^2 = 0.40$$
$$\text{Weighted Average: } 0.24 \left(\frac {14} {25}\right) + 0.40 \left(\frac {11} {25}\right) = 0.31$$

Evaluating the split on **age** using the Gini index:

$$\text{30 or younger: } 1 - \left(\frac {4} {12}\right)^2 - \left(\frac {8} {12}\right)^2 = 0.44$$
$$\text{31 or older: } 1 - \left(\frac {6} {13}\right)^2 - \left(\frac {7} {13}\right)^2 = 0.50$$
$$\text{Weighted Average: } 0.44 \left(\frac {12} {25}\right) + 0.50 \left(\frac {13} {25}\right) = 0.47$$

Again, the decision tree algorithm will try **every possible split** and will choose the one that **reduces the Gini index (and thus increases the "node purity") the most**.

You can think of this as each split increasing the accuracy of predictions. If there is some error at a node, then splitting at that node will result in two nodes with a higher average "node purity" than the original. So, we ensure continually better fits to the training data by continually splitting nodes.

# Classification Error Rate vs. Gini Index

- Gini index is the default in sklearn.
- One advantage Gini index is that it will sometimes make splits that do not improve accuracy but do give better predicted probabilities.

# Summary: Comparing Decision Trees With Other Models

**Advantages of decision trees:**

- They can be used for regression or classification.
- They can be displayed graphically.
- They are highly interpretable.
- They can be specified as a series of rules, and more closely approximate human decision-making than other models.
- Prediction is fast.
- Their features don't need scaling.
- They automatically learn feature interactions.
- Tends to ignore irrelevant features.
- They are non-parametric -- as a result, they will outperform linear models if the relationship between features and response is highly non-linear.

### BONUS
Other splitting criteria to look into (for classification trees)
- [entropy](https://bricaud.github.io/personal-blog/entropy-in-decision-trees/)
- [information gain ratio](https://www3.nd.edu/~rjohns15/cse40647.sp14/www/content/lectures/23%20-%20Decision%20Trees%202.pdf)